Estima de forma aproximada la velocidad angular de rotación de la cámara (grados/segundo) analizando las trayectorias obtenidas por el *tracker* de Lucas-Kanade (ejemplo `LK/lk_track.py`). 

Intentar explicar este programa en un documento juptyer es algo dificil pero el programa en si
es bastante facil
Necesitamos dos cosas:
- Calcular unos puntos/esquinas (p1) en un frame
- Calcular la posicion de los mismos en el frame siguiente.
Una vez que tenemos la posicion de unos puntos en dos instantes de tiempo solo tenemos
que calcular hacia donde se han movido y ya tendriamos nuestro giroscopio

Primero tenemos en encontrar unos puntos para eso usamos


In [1]:
%%script false --no-raise-error
corners = cv.goodFeaturesToTrack(gray, mask=mask, **self.corners_params)
if corners is not None:
    for [(x, y)] in np.float32(corners):
        tracks.append([[x, y]])

Couldn't find program: 'false'


Esto me devuelve una lista con los puntos donde hay una esquina 

Una vez que tenemos los puntos calculamos la nueva posicion de los mismos (p1)

In [2]:
%%script false --no-raise-error
tracks = 
# Sacamos los últimos puntos de cada trayectoria
p0 = np.float32([t[-1] for t in tracks])
# Calculamos la nueva posicion de los puntos p0
p1, _, _ = cv.calcOpticalFlowPyrLK(prevgray, gray, p0, None, **self.lk_params)
# calculamos la posicion en el pasado relativa a p1
p0r, _, _ = cv.calcOpticalFlowPyrLK(gray, prevgray, p1, None, **self.lk_params)

Couldn't find program: 'false'


Para reducir errores solo nos quedamos con los puntos que tienen un futuro es decir
(p1 en t0 ) == p0 en t0   

In [3]:
%%script false --no-raise-error
# el criterio para considerar bueno un punto siguiente es que si lo proyectamos
# hacia el pasado, vuelva muy cerca del punto incial, es decir:
# "back-tracking for match verification between frames"
d = abs(p0 - p0r).reshape(-1, 2).max(axis=1)
good = d < 1
# Elegimos los puntos buenos
new_tracks = []
for t, (x_new, y_new), ok in zip(tracks, p1.reshape(-1, 2), good):
    if not ok:
        continue
    t.append([x_new, y_new])
    if len(t) > self.track_len:
        del t[0]
    new_tracks.append(t)

Couldn't find program: 'false'


Una vez que tenemos los puntos en dos instantes de tiempo p1 y p0 
podemos calcular hacia donde se ha movido la camara esto se hace con
`estimateAffinePartial2D`

In [4]:
import numpy as np
import cv2 as cv
def _get_movement(tracks: list):
    """
    Obtiene t_x y t_y que son "cuanto se ha movido el punto con respecto a su posicion anterior"
    Args:
        tracks (np.):

    Returns:
        t_x: Movimiento en el eje x
        t_y: Movimiento en el eje y

    """
    t_x, t_y = 0, 0
    if len(tracks):
        # Sacamos los ultimos 2 estados
        prev_pts = np.array([t[-2] for t in tracks])
        curr_pts = np.array([t[-1] for t in tracks])

        # Find transformation matrix
        """cos(θ) * s   -sin(θ) * s    t_x
           sin(θ) * s    cos(θ) * s    t_y """
        matriz, inliers = cv.estimateAffinePartial2D(prev_pts, curr_pts)
        # Sacamos el angulo del movimiento respento Y y X | A veces no se puede resolver -_-
        if matriz is not None:
            # theta_X = np.degrees(np.arctan2(-matriz[0, 1], matriz[0, 0]))
            # theta_Y = np.degrees(np.arctan2(-matriz[1, 1], matriz[1, 0]))
            t_x = int(matriz[0, 2])
            t_y = int(matriz[1, 2])

    return t_x, t_y

<video controls src="../img/VROT/vrot_working.mp4" />